In [1]:
from collections import defaultdict
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.metrics import average_precision_score
import sys

sys.path.insert(0, '../Common/')
from COCOWrapper import COCOWrapper
from Dataset import ImageDataset, my_dataloader
from ModelWrapper import ModelWrapper
from ResNet import get_model


In [2]:
data_dir = '/home/gregory/Datasets/objectnet-1.0/images'

modes = {'initial-tune': 'Baseline', 'spire': 'SPIRE', 'fs-3': 'FS'}
trials = [0,1,2,3,4,5,6,7]

coco = COCOWrapper(mode = 'val')

In [3]:
# Get the image labels and locations for ObjectNet
files = []
label_dict = defaultdict(list)

for label_dir in glob.glob('{}/*'.format(data_dir)):
    label = label_dir.split('/')[-1]
    
    filenames = glob.glob('{}/*'.format(label_dir))
    
    for file in filenames:
        files.append(file)
    
    label_dict[label] = filenames
    

In [4]:
# Find the common objects that SPIRE identified as 'main' for some SP
with open('./2-Models/HPS/spire/spire.json', 'r') as f:
    mains = json.load(f)

tmp = []
for main in mains:
    tmp.append(main.replace('+', ' '))
    
mains = tmp

tmp = []
for main in mains:
    if len(label_dict[main]) >= 50:
        tmp.append(main)
        
mains = tmp
print(mains)

['handbag', 'bench', 'tie', 'fork', 'spoon', 'skateboard', 'toothbrush']


In [5]:
# Setup the labels
y = []
for file in files:
    y_tmp = np.zeros((91))
    for main in mains:
        index = coco.get_class_id(main)
        if file in label_dict[main]:
            y_tmp[index] = 1
    y.append(y_tmp)

# Get the model's predictions
out = {}
for mode in modes:
    out_mode = defaultdict(list)
    for trial in trials:
        
        print(mode, trial)
        
        model_dir = './2-Models/Models/{}/trial{}/model.pt'.format(mode, trial)

        model, _ = get_model(mode = 'tune', parent = model_dir, out_features = 91)
        model.eval()
        model.cuda()

        wrapper = ModelWrapper(model)

        dataset = ImageDataset(files, y, objectnet = True)
        dataloader = my_dataloader(dataset)
        y_hat, y_true = wrapper.predict_dataset(dataloader)
        
        for main in mains:
            index = coco.get_class_id(main)
            v = average_precision_score(y_true[:, index], y_hat[:, index])
            out_mode[main].append(v)
    out[mode] = out_mode    

# Get results
print('Results per object')
for main in mains:
    print()
    print('Object: ', main)
    for mode in modes:
        v = out[mode][main]
        print(modes[mode], np.round(np.mean(v), 3), np.round(np.std(v), 4))
        
print()
print()
print('Aggregated results')
print()

for mode in modes:
    tmp = []
    for main in mains:
        tmp.append(np.mean(out[mode][main]))
    print(modes[mode], np.round(np.mean(tmp), 3))

initial-tune 0
initial-tune 1
initial-tune 2
initial-tune 3
initial-tune 4
initial-tune 5
initial-tune 6
initial-tune 7
spire 0
spire 1
spire 2
spire 3
spire 4
spire 5
spire 6
spire 7
fs-3 0
fs-3 1
fs-3 2
fs-3 3
fs-3 4
fs-3 5
fs-3 6
fs-3 7
Results per object

Object:  handbag
Baseline 0.01 0.0026
SPIRE 0.01 0.0026
FS 0.008 0.001

Object:  bench
Baseline 0.034 0.0058
SPIRE 0.033 0.0069
FS 0.03 0.0045

Object:  tie
Baseline 0.01 0.0027
SPIRE 0.014 0.0032
FS 0.01 0.0015

Object:  fork
Baseline 0.013 0.0055
SPIRE 0.026 0.0041
FS 0.02 0.0058

Object:  spoon
Baseline 0.01 0.0019
SPIRE 0.046 0.0086
FS 0.015 0.0068

Object:  skateboard
Baseline 0.006 0.0009
SPIRE 0.008 0.0021
FS 0.006 0.0011

Object:  toothbrush
Baseline 0.042 0.0115
SPIRE 0.047 0.0146
FS 0.043 0.0173


Aggregated results

Baseline 0.018
SPIRE 0.026
FS 0.019
